In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer,StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
import math

In [2]:
def standard(train,test):
    scaler = StandardScaler().fit(train)
    train = scaler.transform(train)
    test = scaler.transform(test)
    return train,test

In [3]:
x_train = pd.read_csv('../data/x_train_30.csv')
y_train = pd.read_csv('../data/pre_y_train.csv')
x_test = pd.read_csv('../data/x_test_30.csv')
x_test_id = pd.read_csv('../data/pre_x_test.csv',usecols=['id'])
#x_train,x_test = standard(x_train,x_test)
#print x_train
#for i in x_train.columns:
#    if x_train[i].dtype != np.int64 and x_train[i].dtype != np.float64:
#        print i

In [4]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1)

In [5]:
params = {}
params['objective'] = 'reg:linear'
params['eval_metric'] = 'rmse'
params['eta'] = 0.02
params['max_depth'] = 6
d_train = xgb.DMatrix(x_train,y_train)
d_valid = xgb.DMatrix(x_valid,y_valid)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgb.train(params, d_train, 1000, watchlist ,early_stopping_rounds=50, verbose_eval=10)

[0]	train-rmse:14.8204	valid-rmse:14.8118
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[10]	train-rmse:12.1136	valid-rmse:12.1064
[20]	train-rmse:9.90265	valid-rmse:9.8966
[30]	train-rmse:8.09701	valid-rmse:8.09196
[40]	train-rmse:6.62273	valid-rmse:6.61834
[50]	train-rmse:5.41939	valid-rmse:5.41555
[60]	train-rmse:4.43767	valid-rmse:4.43432
[70]	train-rmse:3.63731	valid-rmse:3.63454
[80]	train-rmse:2.98538	valid-rmse:2.98365
[90]	train-rmse:2.45508	valid-rmse:2.45427
[100]	train-rmse:2.02467	valid-rmse:2.02499
[110]	train-rmse:1.6764	valid-rmse:1.67814
[120]	train-rmse:1.3959	valid-rmse:1.39898
[130]	train-rmse:1.17129	valid-rmse:1.17573
[140]	train-rmse:0.993003	valid-rmse:0.999008
[150]	train-rmse:0.852874	valid-rmse:0.861011
[160]	train-rmse:0.744209	valid-rmse:0.754459
[170]	train-rmse:0.661249	valid-rmse:0.673745
[180]	train-rmse:0.598941	valid-rmse:0.613629
[190]	train-rmse:0.5527

In [6]:
d_test = xgb.DMatrix(x_test)
y_test = bst.predict(d_test)

In [7]:
sub = pd.DataFrame()
sub['id'] = np.array(x_test_id['id'].values,np.int32)
sub['price_doc'] = np.exp(y_test)-1.0
sub.to_csv('../output/simple11.csv', index=False)